# Solving Burgers' Equation with a physics-informed neural net

---

Let's learn how PINNs work! In this notebook, we'll implement a PINN following one of the [original papers](https://arxiv.org/pdf/1711.10561). This [video](https://www.youtube.com/watch?v=IDIv92Z6Qvc) was also helpful

The goal is to find a pde solution
looks cool: https://arxiv.org/pdf/2403.00599

https://github.com/idrl-lab/PINNpapers


The 1D viscous  Burgers' equation is

$$
\frac{\partial u}{\partial t} + u\frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2} \qquad x \in [-1, 1], \quad t \in [0, 1]
$$

where $u(x, t)$ gives the wave displacement and $\nu$ is the diffusion coefficient, describing

We will solve it with the Dirichlet boundary conditions 

$$
u(-1,t)=u(1,t)=0
$$

and initial conditions

$$
u(x,0) = - \sin(\pi x).
$$

First, we need to rewrite the PDE and its boundary / initial conditions into the form

\begin{align*}
\mathcal{D}[u(x, t)] &= f(x, t), \quad x \in \Omega \subset \mathbb{R}^d \\
B_k[u(x, t)] &= g_k(x, t), \quad x \in \Gamma_k \subset \partial \Omega \\
I_l[u(x, t=0)] &= h_l(x, t=0), \quad x \in \Omega \subset \mathbb{R}^d \\
\end{align*}

as described in [this notebook](./spring.ipynb). The PDE becomes


$$
\left( \frac{\partial }{\partial t} + u\frac{\partial }{\partial x} - \nu \frac{\partial^2 }{\partial x^2} \right) \big[u \big] - 0 = 0 
$$

The boundary conditions remain the same since they equal zero, but we will explicitly write them out. So the boundary / initial conditions are

\begin{align*}
1)& \quad u(-1,t) = 0 \quad \Rightarrow \quad u(-1,t) - 0 = 0 \\
2)& \quad u(1,t) = 0 \quad \Rightarrow \quad u(1,t) - 0 = 0 \\
3)& \quad u(x,t_\text{min}) = - \sin(\pi x) \quad \Rightarrow \quad u(x,t_\text{min}) + \sin(\pi x) = 0 \\

\end{align*}

The PINN is trained to directly approximate the solution to the differential equation

$$
N\!N(x, t;\theta) \approx u(x, t)
$$





---

### Building the loss function

The supervised **boundary loss** $ L_B(\theta)$ is given by
$$
L_B(\theta) = \sum_{k} \frac{\lambda_k}{N_{Bk}} \sum_{j}^{N_{Bk}} \left\| B_k \left[ NN(x_{kj}, t_j; \theta) \right] - g_k(x_{kj}, t_j) \right\|^2 
$$

where the outer summation with index $k$ is over the various given boundary conditions $B_k$ on the PDE. $N_{Bk}$ is the number of *boundary* training points $x_{kj}$ for boundary condition $B_k$. Then the inner summation over $j$ sums the $N_{Bk}$ squared residuals, and $\lambda_k$ is the regularization coefficient for the corresponding boundary condition.

The supervised **initial condition loss** $ L_I(\theta)$ is given by
$$
L_I(\theta) = \sum_{l} \frac{\lambda_l}{N_{Il}} \sum_{j}^{N_{Il}} \left\| I_l \left[ NN(x_{lj}, t_j; \theta) \right] - h_l(x_{lj}, t_j) \right\|^2 
$$

with the same idea as above except that now $t_j$ is really $t_\text{min}$, the initial time.

The unsupervised **physics loss** $ L_P(\theta)$ is given by 
$$
L_P(\theta) = \frac{1}{N_P} \sum_{i}^{N_P} \left\| \mathcal{D} \left[ NN(x_i, t_i; \theta) \right] - f(x_i, t_i) \right\|^2 
$$

where we sum the $N_P$ squared residuals.

Then our regularized loss function is made up of the physics loss, $L_P(\theta)$, the boundary loss, $L_B(\theta)$, and the initial condition loss, $L_I(\theta)$, with $t_\text{min} = 0$


\begin{align*}
\mathcal{L}(\theta) &= \lambda_P L_P(\theta) + \sum_k \lambda_{Bk} L_{Bk}(\theta) + \lambda_I L_I(\theta) \\
&= \frac{\lambda_P}{N_p} \sum_{i=1}^{N_P} \left( \left[\frac{\partial }{\partial t} + N\!N(x_i, t_i; \theta) \frac{\partial }{\partial x} - \nu \frac{\partial^2 }{\partial x^2} \right] N\!N (x_i, t_i; \theta)\right)^2 \\
&+ \frac{\lambda_{B1}}{N_{B1}} \sum_{j=1}^{N_{B1}} \left( N\!N(-1, t_j; \theta) - 0 \right)^2 + \frac{\lambda_{B2}}{N_{B2}} \sum_{j=1}^{N_{B2}} \left(N\!N(1, t_j; \theta) - 0\right)^2 \\
&+ \frac{\lambda_I}{N_I} \sum_{k=1}^{N_I} \left( N\!N(x_k, 0; \theta) + \sin(\pi x_k) \right)^2
\end{align*}

which we can simplify to

\begin{align*}
\mathcal{L}(\theta) &= \lambda_P L_P(\theta) + \sum \lambda_B L_B(\theta) + \lambda_I L_I(\theta) \\
&= \frac{\lambda_P}{N_p} \left\| \left[\frac{\partial }{\partial t} + N\!N(x_i, t_i; \theta) \frac{\partial }{\partial x} - \nu \frac{\partial^2 }{\partial x^2} \right] N\!N (x_i, t_i; \theta) \right\|^2\\
&+ \frac{\lambda_{B1}}{N_{B1}} \left\| N\!N(-1, t_j; \theta) \right\|^2 + \frac{\lambda_{B2}}{N_{B2}} \left\|N\!N(1, t_j; \theta) \right\|^2 \\
&+ \frac{\lambda_I}{N_I} \left\| N\!N(x_k, 0; \theta) + \sin(\pi x_k) \right\|^2
\end{align*}

---

### Adding a transform layer

It looks like we have a few regularization hyperparameters to choose... one way we can remove some of these is by adding a transform function after the output layer of the network. This transform layer would enforce the initial condition at $t_\text{min}$ and look something like

```py
def transform_output(x, t, u, Tmin:, h):
    return h(x, t) + u(x, t) * (t - Tmin)
```

where `x` and `t` are the inputs to the PINN, `u` is the PINN output approximation of the PDE, `Tmin` is the initial condition time, and `h` is the initial condition function itself. As an equation,

$$
u_{\text{transformed}}(x, t) = h(x, t) + NN(x, t; \theta) \cdot (t - t_{\text{min}})
$$

where, again, we have the PDE solution approximation $N\!N(x, t;\theta) \approx u(x, t)$. The idea behind this is simple: the transformation ensures that our initial condition function $h(x, t)$ is satisfied when $t = t_\text{min}$. In all other cases, when $t > t_\text{min}$, we see that the output is a combination of the specified initial condition function and the untransformed network output. 

So by using this transform layer, we get to remove the initial condition loss term from our overall loss function -- simplifying it and giving us one less hyperparameter to tune :)

However, the transform layer has no learnable parameters and is technically not part of the neural network. But since it sits at the end of the network, it will have a strong influence on its output and, thus, the loss. This means that errors backpropagating through the network will tune its weights with information about the initial conditions recieved not explicitly from the loss function, but from the transform layer itself.

If we consider the transform layer as part of the network, we can write our new PDE solution approximation as
$$
N\!N'(x, t;\theta) \approx u_{\text{transformed}}(x, t, NN(x, t; \theta))
$$

where $N\!N'$ is the output of the network *with* the transform layer, and $N\!N$ is the previous vanilla output (included explicitly in the transformed solution here).

Then our new loss function becomes

\begin{align*}
\mathcal{L}(\theta) &= \lambda_P L_P(\theta) + \sum \lambda_B L_B(\theta)  \\
&= \frac{\lambda_P}{N_p} \left\| \left[\frac{\partial }{\partial t} + N\!N'(x_i, t_i; \theta) \frac{\partial }{\partial x} - \nu \frac{\partial^2 }{\partial x^2} \right] N\!N' (x_i, t_i; \theta) \right\|^2\\
&+ \frac{\lambda_{B1}}{N_{B1}} \left\| N\!N'(-1, t_j; \theta) \right\|^2 + \frac{\lambda_{B2}}{N_{B2}} \left\|N\!N'(1, t_j; \theta) \right\|^2 
\end{align*}

with final layer
$$
u_{\text{transformed}}(x, t) = \sin(\pi x) + NN(x, t; \theta) \cdot (t - t_{\text{min}})
$$


In [2]:
import torch
import torch.nn as nn